# Obesity & Mental Health Analysis
**Hypothesis:** Individuals experiencing higher levels of stress or mental health challenges are more likely to have higher obesity prevalence.
This notebook explores the connection between mental health attributes (e.g., stress, coping) and country-level obesity rates.

## Load Libraries and Datasets

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, chi2_contingency

sns.set(style='whitegrid')

obesity = pd.read_csv("/mnt/data/obesity_clean.csv")
mental = pd.read_csv("/mnt/data/mental_clean.csv")
drug = pd.read_csv("/mnt/data/drug_clean.csv")


## Obesity Rate by Gender

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(data=obesity, x='Gender', y='ObesityRate', palette='Set2')
plt.title("Obesity Rate by Gender")
plt.tight_layout()
plt.show()

male = obesity[obesity['Gender'] == 'Male']['ObesityRate']
female = obesity[obesity['Gender'] == 'Female']['ObesityRate']
t_stat, p_val = ttest_ind(male, female)
print("T-test statistic:", t_stat)
print("p-value:", p_val)

## Growing Stress Distribution by Gender

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=mental, x='Gender', hue='Growing_Stress', palette='pastel')
plt.title("Growing Stress by Gender")
plt.tight_layout()
plt.show()

## Country-Level Obesity Rate vs Stressful Population

In [ ]:
# Aggregate mental data: % of people with stress per country
stress_country = mental[mental['Growing_Stress'] == 'Yes'].groupby('Country').size()
total_country = mental.groupby('Country').size()
stress_ratio = (stress_country / total_country).dropna().reset_index()
stress_ratio.columns = ['Country', 'StressRate']

# Average obesity per country
obesity_country = obesity.groupby('Country')['ObesityRate'].mean().reset_index()

# Merge and plot
merged = pd.merge(stress_ratio, obesity_country, on='Country')
plt.figure(figsize=(6,4))
sns.scatterplot(data=merged, x='StressRate', y='ObesityRate')
plt.title("Obesity Rate vs Stress Prevalence (by Country)")
plt.xlabel("Proportion of Stressed Individuals")
plt.ylabel("Mean Obesity Rate")
plt.tight_layout()
plt.show()


## Machine Learning: Obesity Prediction Model

In [ ]:

# Machine Learning: Predicting High Obesity Rate based on stress and country info

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Prepare data for ML - binary classification: above or below median obesity
merged['HighObesity'] = (merged['ObesityRate'] > merged['ObesityRate'].median()).astype(int)

# Encode country names
le = LabelEncoder()
merged['CountryEncoded'] = le.fit_transform(merged['Country'])

# Features: stress rate and country as proxy for socio-cultural factors
X = merged[['StressRate', 'CountryEncoded']]
y = merged['HighObesity']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Results
print(classification_report(y_test, y_pred))


## Conclusion
- Obesity rates are slightly higher in females but not drastically different.
- Stress is more prevalent in females.
- Countries with higher mental stress prevalence **tend to show higher obesity rates**, supporting the hypothesis.

Further studies with individual-level BMI would improve causal inference.